# Import Libraries

In [1]:
import os
import ipywidgets as widgets
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from typing import List
from IPython.display import display, HTML
from data.loader import DatasetLoader

# Load Datasets

In [2]:
dataset_directory = 'data/datasets'
extracted_exchanges = set()
extracted_symbols = set()
dataset_configs = []

for filename in os.listdir(path=dataset_directory):
    exchange, symbol, _ = filename.split('_')
    extracted_exchanges.add(exchange)
    extracted_symbols.add(symbol)
    dataset_configs.append({'symbol': symbol, 'exchange': exchange})
        
f'Extracted {len(dataset_configs)} datasets of {len(extracted_exchanges)} unique exchanges and {len(extracted_symbols)} unique symbols'

'Extracted 151 datasets of 14 unique exchanges and 46 unique symbols'

In [3]:
dataframes = DatasetLoader(dataset_directory=dataset_directory).load_datasets(dataset_configs=dataset_configs)
dataframes['date'] = dataframes['date'].astype(str).apply(lambda d: d.split(' ')[0])
dataframes.groupby(by=['exchange', 'symbol']).head(1)

,exchange,symbol,date,open,high,low,close,volume,ema,vwap,rsi,cci,adx,day,targets
0,Abucoins,BTCUSD,2017-11-21,8225.410000,8414.870000,7898.14000,8132.54000,3.059447e+04,7516.706981,6954.421359,65.112345,139.740506,0.0,1,-0.011355
0,Binance,AAVEUSDT,2020-11-03,29.723000,30.075000,27.42400,28.12200,2.391137e+06,32.998003,34.604850,31.018244,-119.882237,0.0,1,-0.055403
0,Binance,ADAUSDT,2018-05-06,0.365020,0.366410,0.33538,0.34654,2.220280e+07,0.335429,0.336108,60.586776,63.450307,0.0,6,-0.051954
0,Binance,ATOMUSDT,2019-05-18,4.413000,5.000000,4.33000,4.68400,8.791709e+06,4.278062,4.348880,60.244497,59.266228,0.0,5,0.059598
0,Binance,AVAXUSDT,2020-10-11,4.020500,4.176200,3.93330,4.09600,2.203938e+06,4.096434,4.017947,45.017671,-15.584925,0.0,6,0.017983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Yobit,BTCUSD,2017-06-14,2772.140000,2822.000000,2476.87000,2580.71000,1.554554e+05,2682.292936,2703.519150,49.342972,19.347045,0.0,2,-0.071555
0,Yobit,DOGEUSD,2017-06-14,0.003383,0.003383,0.00300,0.00300,1.187070e+03,0.003217,0.003299,47.288654,8.415431,0.0,2,-0.120151
0,Yobit,ETHUSD,2017-06-14,401.000000,401.150000,342.53000,362.00000,4.989309e+04,305.053234,330.955990,76.209985,132.433611,0.0,2,-0.102317
0,Yobit,LTCUSD,2017-06-14,31.500000,32.000000,29.50000,29.50000,8.509000e+02,29.534827,29.631351,50.901292,72.043264,0.0,2,-0.065597


# Visualizing Candlesticks

In [4]:
def visualize_candlesticks(symbol: str, exchanges: List[str]):
    if not isinstance(exchanges, list):
        exchanges = list(exchanges)    
    
    mask = (dataframes['symbol'] == symbol) & (dataframes['exchange'].isin(exchanges))
    symbol_df = dataframes.loc[mask, ['symbol', 'exchange', 'date', 'open', 'high', 'low', 'close']]
    
    if symbol_df.shape[0] > 0:
        total_exchanges = symbol_df['exchange'].unique().tolist()
        num_exchanges = len(total_exchanges)
        palette = sns.color_palette('hsv', num_exchanges).as_hex()
        color_dict = dict(zip(total_exchanges, palette))
        
        traces = []
        for exchange in total_exchanges:
            data = symbol_df[symbol_df['exchange'] == exchange]
            traces.append(go.Candlestick(
                x=data['date'],
                open=data['open'],
                high=data['high'],
                low=data['low'],
                close=data['close'],
                name=exchange,
                increasing=dict(line=dict(color=color_dict[exchange])),
                decreasing=dict(line=dict(color=color_dict[exchange]))
            ))
            

        layout = go.Layout(
            title=f'Candlestick Chart of {symbol} in {num_exchanges}',
            xaxis=dict(title='Date', rangeslider=dict(visible=False))
        )
        fig = go.Figure(data=traces, layout=layout)
        fig.show()
        display(HTML(symbol_df.head(5).to_html()))
        return fig
    return None


symbol_dropdown = widgets.Dropdown(
    options=sorted(extracted_symbols),
    value='BTCUSDT',
    description='Symbol Pair:',
    disabled=False
)
exchange_box = widgets.SelectMultiple(
    options=sorted(extracted_exchanges),
    value=list(extracted_exchanges),
    description='Exhanges',
    disabled=False
)
hbox = widgets.HBox([symbol_dropdown, exchange_box])

candlesticks_fig_output = widgets.interactive_output(
    visualize_candlesticks, 
    {'symbol': symbol_dropdown, 'exchanges': exchange_box}
)
display(hbox, candlesticks_fig_output)

Output()